In [203]:
# 라이브러리 모음
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import csv
import os
import requests
import glob
import pandas as pd

In [ ]:
# 포켓몬 도감을 만들자
# 예를 들어, 포켓몬 151마리를 저장!!
# 포켓몬 폴더 만들어서 이미지 151개 저장!!
# 각 이미지를 클릭하면 -> 포켓몬 정보 페이지
# 정보 페이지
# 1. 포켓몬 이름
# 2. 포켓몬 타입 (단, 2개 이상일수도 있음을 유의!!)
# 3. 가져오고 싶은 추가 정보들(설명, 키, 몸무게, 분류)
# 4. 이 정보를 토대로 Pokemon class를 설계
# 5. pokemon_list에 class를 통해 생성된 instance저장!!
# 이 정보를 토대로 DataFrame
# 이미지 경로 매칭
# 클릭 -> 포켓몬 이미지 뜨게

In [226]:
# 창 열기

poke_url = "https://pokemonkorea.co.kr/pokedex"
driver = wb.Chrome()
driver.get(poke_url)

In [ ]:
send_text = input("도감명을을 입력하세요 :")

if not os.path.isdir(send_text):
    os.mkdir(send_text)
else:
    print("해당 도감이 존재합니다.")

In [ ]:
# 정보창에서 이동
amount = int(input("원하는 포켓몬의 숫자를 입력하세요 :"))

for i in range(amount):
    move_next = driver.find_element(By.CLASS_NAME, "icon-next-book")
    move_next.click()
    time.sleep(0.1)

In [ ]:
# 아래 스크롤 하기

amount = int(input("원하는 사진의 숫자를 입력하세요 :"))
scroll = amount // 18

for i in range(scroll):
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time.sleep(1)

# 설정 양보다 이미지가 적을 경우
# if len(p_images) < amount:
#   amount = len(p_images)

In [9]:
# 이미지 가져오기

p_images = driver.find_elements(By.CLASS_NAME, "img-fluid")
print(len(p_images))
p_names = driver.find_elements(By.CSS_SELECTOR, ".bx-txt h3")
print(len(p_names))

180
180


In [227]:
# 원하는 시작 위치
p_data = driver.find_elements(By.CLASS_NAME, "tumb-wrp")
p_data[0].click()

In [287]:
# 이미지 + 정보 가져오기
p_images = driver.find_element(By.CLASS_NAME, "col-lg-6.col-12")

p_num_name = driver.find_element(By.CSS_SELECTOR, ".col-lg-6.col-12 h3")
p_num_name = p_num_name.text.split("\n")
p_num = p_num_name[0]
if len(p_num_name) == 3:
    p_name = p_num_name[1] +" " + p_num_name[2]
else:
    p_name = p_num_name[1]
p_name = p_name.replace("/", ", ")
    
p_types = driver.find_element(By.CLASS_NAME, "d-flex")
p_types = p_types.text.replace("\n", ", ")

p_cates = driver.find_elements(By.CSS_SELECTOR, ".bx-detail .row:first-child .col-4 p")
p_cates = p_cates[len(p_cates)-1].text

char_btn = driver.find_elements(By.CLASS_NAME, "btn-char.no-style")
print(len(char_btn))
for i in range(0, len(char_btn)):
    char_btn[i].click()

    time.sleep(0.2)
    p_char = driver.find_element(By.CLASS_NAME, "modal-title")
    print(p_char.text)

    p_char_explain = driver.find_element(By.ID, "exampleModalLabel2")
    print(p_char_explain.text)

    time.sleep(0.6)
    char_close_btn = driver.find_element(By.CLASS_NAME, "btn-close2.close")
    char_close_btn.click()

1
조가비갑옷
단단한 껍질의 보호를 받아 상대의 공격이 급소에 맞지 않는다.


In [ ]:
for i in range(amount):
    img_url = p_images[i].get_attribute("src")
    time.sleep(0.1)
    # gif 예외처리(오류유발)
    if "gif" in img_url:
        continue

    # 경로를 통해 이미지 정보 가져오기
    response = requests.get(img_url)
    img_data = response.content
    img_path = f"{send_text}/{send_text}{i+1}.jpg"
    # 이미지 저장
    with open(img_path, "wb") as file:
        file.write(img_data)
        print(f"{send_text}{i+1} 저장 완료")

driver.close()
    

In [288]:
class poke_pokedex:
    def __init__(self, images, num, name, types, cates, char, char_explain):
        self.images = images
        self.num = num
        self.name = name
        self.types = types
        self.cates = cates
        self.char = char
        self.char_explain = char_explain

    def show_poke_info(self):
        if len(self.char) >= 2:
            for i in range(len(self.char)):
                print(f"num - {self.num} / name - {self.name} / types - {self.types} / cates - {self.cates} / char - {self.char[i]} / char_explain - {self.char_explain[i]}")
        else:
            print(f"num - {self.num} / name - {self.name} / types - {self.types} / cates - {self.cates} / char - {self.char} / char_explain - {self.char_explain}")  

    def poke_to_csv(self):
        return [self.num,self.name,self.types,self.cates,self.char,self.char_explain]
    

num - No. 0004 / name - 파이리 / types - 불꽃 / cates - 도롱뇽포켓몬 / char - 맹화 / char_explain - HP가 줄었을 때 불꽃타입 기술의 위력이 올라간다.


In [ ]:
# 창 열기

poke_url = "https://pokemonkorea.co.kr/pokedex"
driver = wb.Chrome()
driver.get(poke_url)

send_text = input("도감명을을 입력하세요 :")

if not os.path.isdir(send_text):
    os.mkdir(send_text)
    os.mkdir(f"{send_text}\images")
    os.mkdir(f"{send_text}\infomation")
else:
    print("해당 도감이 존재합니다.")

# 정보창에서 이동
amount = int(input("원하는 포켓몬의 숫자를 입력하세요 :"))

# 원하는 시작 위치
p_data = driver.find_elements(By.CLASS_NAME, "tumb-wrp")
p_data[0].click()

result = []
p_char_list = []
p_char_explain_list = []

for i in range(amount):
    # 이미지 + 정보 가져오기
    p_images = driver.find_element(By.CSS_SELECTOR, ".col-lg-6.col-12>img")

    p_num_name = driver.find_element(By.CSS_SELECTOR, ".col-lg-6.col-12 h3")
    p_num_name = p_num_name.text.split("\n")
    p_num = p_num_name[0]
    if len(p_num_name) == 3:
        p_name = p_num_name[1] +" " + p_num_name[2]
    else:
        p_name = p_num_name[1]
    p_name = p_name.replace("/", ", ")
        
    p_types = driver.find_element(By.CLASS_NAME, "d-flex")
    p_types = p_types.text.replace("\n", ", ")

    p_cates = driver.find_elements(By.CSS_SELECTOR, ".bx-detail .row:first-child .col-4 p")
    p_cates = p_cates[len(p_cates)-1].text

    char_btn = driver.find_elements(By.CLASS_NAME, "btn-char.no-style")
    for i in range(0, len(char_btn)):
        char_btn[i].click()

        time.sleep(0.2)
        p_char = driver.find_element(By.CLASS_NAME, "modal-title")
        p_char_list.append(p_char.text)

        p_char_explain = driver.find_element(By.ID, "exampleModalLabel2")
        p_char_explain_list.append(p_char_explain.text)

        time.sleep(0.6)
        char_close_btn = driver.find_element(By.CLASS_NAME, "btn-close2.close")
        char_close_btn.click()

    img_url = p_images.get_attribute("src")
    response = requests.get(img_url)
    img_data = response.content
    img_path = f"{send_text}/images/{p_num} {p_name}.jpg"
    with open(img_path, "wb") as file:
        file.write(img_data)
        print(f"{p_name} 저장 완료")

    a = poke_pokedex(p_images, p_num, p_name, p_types, p_cates, p_char, p_char_explain)
    result.append(a)

    time.sleep(0.2)
    move_next = driver.find_element(By.CLASS_NAME, "icon-next-book")
    move_next.click()

driver.close()

for r in result:
    r.show_poke_info()

with open(f"{send_text}/infomation/{send_text}.csv", "w", newline="") as file:
        write = csv.writer(file)
        for i in range(amount):
                write.writerow(result[i].poke_to_csv())
                


해당 도감이 존재합니다.
이상해씨 저장 완료
이상해풀 저장 완료
이상해꽃 저장 완료
메가이상해꽃 저장 완료
이상해꽃 거다이맥스의 모습 저장 완료
파이리 저장 완료
리자드 저장 완료
리자몽 저장 완료
메가리자몽X 저장 완료
메가리자몽Y 저장 완료
리자몽 거다이맥스의 모습 저장 완료
꼬부기 저장 완료
어니부기 저장 완료
거북왕 저장 완료
메가거북왕 저장 완료
거북왕 거다이맥스의 모습 저장 완료
캐터피 저장 완료
단데기 저장 완료
버터플 저장 완료
버터플 거다이맥스의 모습 저장 완료
뿔충이 저장 완료
딱충이 저장 완료
독침붕 저장 완료
메가독침붕 저장 완료
구구 저장 완료
피죤 저장 완료
피죤투 저장 완료
메가피죤투 저장 완료
꼬렛 저장 완료
꼬렛 알로라의 모습 저장 완료
레트라 저장 완료
레트라 알로라의 모습 저장 완료
깨비참 저장 완료
깨비드릴조 저장 완료
아보 저장 완료
아보크 저장 완료
피카츄 저장 완료
피카츄 거다이맥스의 모습 저장 완료
라이츄 저장 완료
라이츄 알로라의 모습 저장 완료
모래두지 저장 완료
모래두지 알로라의 모습 저장 완료
고지 저장 완료
고지 알로라의 모습 저장 완료
니드런♀ 저장 완료
니드리나 저장 완료
니드퀸 저장 완료
니드런♂ 저장 완료
니드리노 저장 완료
니드킹 저장 완료
삐삐 저장 완료
픽시 저장 완료
식스테일 저장 완료
식스테일 알로라의 모습 저장 완료
나인테일 저장 완료
나인테일 알로라의 모습 저장 완료
푸린 저장 완료
푸크린 저장 완료
주뱃 저장 완료
골뱃 저장 완료
뚜벅쵸 저장 완료
냄새꼬 저장 완료
라플레시아 저장 완료
파라스 저장 완료
파라섹트 저장 완료
콘팡 저장 완료
도나리 저장 완료
디그다 저장 완료
디그다 알로라의 모습 저장 완료
닥트리오 저장 완료
닥트리오 알로라의 모습 저장 완료
나옹 저장 완료
나옹 알로라의 모습 저장 완료
나옹 가라르의 모습 저장 완료
나옹 거다이맥스의 모습 저장 완료
페르시온 저장 완료
페르시온 알로라의 모습 저장 완료
고라파덕 저장 완료
골덕 저장 완료
망키 저장 완료
성원숭 저장 완료

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=132.0.6834.197)
Stacktrace:
	GetHandleVerifier [0x00007FF779FA02F5+28725]
	(No symbol) [0x00007FF779F02AE0]
	(No symbol) [0x00007FF779D94F39]
	(No symbol) [0x00007FF779DEA7D4]
	(No symbol) [0x00007FF779DDC6B4]
	(No symbol) [0x00007FF779E0FFAA]
	(No symbol) [0x00007FF779DDBF86]
	(No symbol) [0x00007FF779E101C0]
	(No symbol) [0x00007FF779E30181]
	(No symbol) [0x00007FF779E0FD53]
	(No symbol) [0x00007FF779DDA0E3]
	(No symbol) [0x00007FF779DDB471]
	GetHandleVerifier [0x00007FF77A2CF30D+3366989]
	GetHandleVerifier [0x00007FF77A2E12F0+3440688]
	GetHandleVerifier [0x00007FF77A2D78FD+3401277]
	GetHandleVerifier [0x00007FF77A06AAAB+858091]
	(No symbol) [0x00007FF779F0E74F]
	(No symbol) [0x00007FF779F0A304]
	(No symbol) [0x00007FF779F0A49D]
	(No symbol) [0x00007FF779EF8B69]
	BaseThreadInitThunk [0x00007FFB235FE8D7+23]
	RtlUserThreadStart [0x00007FFB24F3BF2C+44]


In [ ]:
class test_c:
    def __init__(self, test_i, test_l):
        self.test_i = test_i
        self.test_l = []